In [1]:
"""
@author: pho
"""
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False
import sys
import importlib
import numpy as np
import pandas as pd
from pathlib import Path
import h5py
import hdf5storage # conda install hdf5storage

from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass
from neuropy.core import DataWriter, NeuronType, Neurons, BinnedSpiketrain, Mua, ProbeGroup, Position, Epoch, Signal, Laps, FlattenedSpiketrains


from pyphocorehelpers.print_helpers import print_keys_if_possible, debug_dump_object_member_shapes
# # import PhoPositionalData as pdp
# # from pyphoplacecellanalysis.PhoPositionalData. import load_exported, process_data
# from pyphoplacecellanalysis.PhoPositionalData.load_exported import *
# # from pyphoplacecellanalysis.PhoPositionalData.process_data import process_positionalAnalysis_data, gen_2d_histrogram, get_heatmap_color_vectors, process_chunk_equal_poritions_data, extract_spike_timeseries
# from pyphoplacecellanalysis.PhoPositionalData.process_data import *
# from pyphoplacecellanalysis.PhoPositionalData.plot_data import *
# from pyphoplacecellanalysis.PhoPositionalData.import_data import *
# from pyphoplacecellanalysis.PhoPositionalData.load_exported import import_mat_file
# from pyphoplacecellanalysis.PhoPositionalData.process_data import process_positionalAnalysis_data, gen_2d_histrogram, get_heatmap_color_vectors, process_chunk_equal_poritions_data, extract_spike_timeseries
# from pyphoplacecellanalysis.PhoPositionalData.process_data import process_positionalAnalysis_data, extract_spike_timeseries

In [2]:
## Drastically improve Python Tracebacks on exceptions:
# from rich.traceback import install
# install()

## Automatic debugger on exception raised
%pdb on

Automatic pdb calling has been turned ON


In [11]:
# RoyMaze1:
# mat_import_parent_path = Path(r'R:\data\RoyMaze1')
mat_import_parent_path = Path(r'R:\rMBP Python Repos 2022-07-07\PhoNeuronGillespie2021CodeRepo\PhoMatlabDataScripting\ExportedData\RoyMaze1')
# mat_import_parent_path = Path(r'C:\Share\data\RoyMaze1') # Old one
# mat_import_file = r'C:\Share\data\RoyMaze1\ExportedData.mat'
# all_vars = load_position_spikes_extras_mats(parent_path=mat_import_parent_path)
all_vars = HiroDataSessionFormatRegisteredClass._load_all_mats(parent_path=mat_import_parent_path)
# all_vars

Loading matlab import file results to R:\rMBP Python Repos 2022-07-07\PhoNeuronGillespie2021CodeRepo\PhoMatlabDataScripting\ExportedData\RoyMaze1\ExportedData\positionAnalysis.mat... done.
Loading matlab import file results to R:\rMBP Python Repos 2022-07-07\PhoNeuronGillespie2021CodeRepo\PhoMatlabDataScripting\ExportedData\RoyMaze1\ExportedData\spikesAnalysis.mat... done.
Loading matlab import file results to R:\rMBP Python Repos 2022-07-07\PhoNeuronGillespie2021CodeRepo\PhoMatlabDataScripting\ExportedData\RoyMaze1\ExportedData\extrasAnalysis.mat... done.
num_periods: 668


In [12]:
session = HiroDataSessionFormatRegisteredClass.build_session(basedir=mat_import_parent_path)
session

DataSession(config: <SessionConfig: {'basepath': WindowsPath('R:/rMBP Python Repos 2022-07-07/PhoNeuronGillespie2021CodeRepo/PhoMatlabDataScripting/ExportedData/RoyMaze1'), 'session_name': 'RoyMaze1', 'session_spec': <SessionFolderSpec: {'required_files': [], 'optional_files': [], 'additional_validation_requirements': []};>, 'is_resolved': True, 'resolved_required_filespecs_dict': {}, 'resolved_optional_filespecs_dict': {}};>): Not yet configured.

In [5]:
# all_vars.extras
# print_keys_if_possible('pos', all_vars.pos)
print_keys_if_possible('extras', all_vars.extras)
# print_keys_if_possible('all_vars', all_vars)
# debug_dump_object_member_shapes(all_vars)

- extras: <class 'neuropy.utils.dynamic_container.DynamicContainer'>
	- behavioral_periods: <class 'pandas.core.frame.DataFrame'> - (668, 6)
	- behavioral_epochs: <class 'pandas.core.frame.DataFrame'> - (3, 7)
	- num_periods: <class 'int'>


In [6]:
type(all_vars.extras.behavioral_epochs) # pandas.core.frame.DataFrame
print(list(all_vars.extras.behavioral_epochs.columns)) # ['epoch_index', 'start_seconds_absolute', 'end_seconds_absolute', 'start_seconds', 'end_seconds', 'duration']
all_vars.extras.behavioral_epochs

['epoch_index', 'start_seconds_absolute', 'end_seconds_absolute', 'start_seconds', 'end_seconds', 'duration', 'label']


,epoch_index,start_seconds_absolute,end_seconds_absolute,start_seconds,end_seconds,duration,label
0,0.0,68368.714228,79168.671561,0.000000,10799.957333,10799.957333,pre_sleep
1,1.0,79368.012194,90198.911394,10999.297966,21830.197166,10830.899200,track
2,2.0,92919.765028,103719.773561,24551.050800,35351.059333,10800.008533,post_sleep


In [13]:
## Adds Session.paradigm (Epochs)
session_absolute_start_timestamp = all_vars.extras.behavioral_epochs.loc[0, 'start_seconds_absolute'] # 68368.714228
session.config.absolute_start_timestamp = session_absolute_start_timestamp
# 'start_seconds' and 'end_seconds' are relative to start
if 'label' in all_vars.extras.behavioral_epochs.columns:
    epoch_labels = all_vars.extras.behavioral_epochs['label']
else:
    num_rows = all_vars.extras.behavioral_epochs.shape[0]
    epoch_labels = [f'epoch{i}' for i in np.arange(num_rows)]
epochs_df = pd.DataFrame({'label':epoch_labels, 'start':all_vars.extras.behavioral_epochs['start_seconds'].to_numpy(),'stop':all_vars.extras.behavioral_epochs['end_seconds'].to_numpy()})
session.paradigm = Epoch(epochs=epochs_df)
session.paradigm

        label         start          stop      duration
0   pre_sleep      0.000000  10799.957333  10799.957333
1       track  10999.297966  21830.197166  10830.899200
2  post_sleep  24551.050800  35351.059333  10800.008533

In [14]:
## Adds Positions:
position_sampling_rate_Hz = 1.0 / np.nanmean(np.diff(all_vars.pos.t)) # 1.0/0.03336651239320582 = 29.97016853950917 Hz
session.config.position_sampling_rate_Hz = position_sampling_rate_Hz
# session.position = Position(traces=np.vstack((x, y)), computed_traces=np.full([1, num_samples], np.nan), t_start=active_t_start, sampling_rate=position_sampling_rate_Hz)
t_rel = all_vars.pos.t
x = all_vars.pos.x
y = all_vars.pos.y
session.position = Position.from_separate_arrays(t_rel, x, y)
session.position

<Position: {'_filename': None, '_metadata': None, '_data':                    t      x      y
0       10999.801938  641.0  214.5
1       10999.835862  641.0  214.5
2       10999.869332  641.0  214.5
3       10999.902526  641.0  214.5
4       10999.936029  641.0  214.5
...              ...    ...    ...
324569  21829.537433  623.5  219.0
324570  21829.570327  623.5  219.0
324571  21829.604142  623.5  219.0
324572  21829.637670  623.5  219.0
324573  21829.670965  623.5  219.0

[324574 rows x 3 columns]};>

In [16]:
## Adds Spikes:
print(list(all_vars.spikes.keys())) # ['spike_matrix', 'spike_cells', 'num_cells', 'spike_list', 'spike_positions_list', 'flat_cell_ids', 'reverse_cellID_idx_lookup_map', 'spikes_cell_info_out_dict']

In [26]:
all_vars.spikes.spike_list[0].shape # (293076,)
all_vars.spikes.spike_list[1].shape # (9297,)

# len(all_vars.spikes.spike_list) # 126

In [27]:
session.paradigm.t_stop

35351.059333

In [16]:
# all_vars.spikes
# 'spike_cells'
all_vars.spikes.spikes_cell_info_out_dict

['spike_matrix', 'spike_cells', 'num_cells', 'spike_list', 'spike_positions_list', 'flat_cell_ids', 'reverse_cellID_idx_lookup_map', 'spikes_cell_info_out_dict']


DynamicContainer({'shank': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 8], dtype=uint8), 'cluster': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,
       13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
       30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42,  1,  2,  3,  4,
        5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,  1,  2,
        3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,  1,  2,  3,  4,
        5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
  

In [19]:
flat_cell_ids = all_vars.spikes.spikes_cell_info_out_dict.aclu
cell_type = NeuronType.from_qclu_series(qclu_Series=all_vars.spikes.spikes_cell_info_out_dict.qclu)
shank_ids = all_vars.spikes.spikes_cell_info_out_dict.shank
# cluster_ids = all_vars.spikes.spikes_cell_info_out_dict.cluster # NOT USED

In [30]:
spiketrains = np.array(all_vars.spikes.spike_list, dtype='object')
# t_stop = np.max(flat_spikes_out_dict[time_variable_name])
t_stop = session.paradigm.t_stop
flat_cell_ids = np.array(flat_cell_ids)
# all_vars.spikes.spikes_cell_info_out_dict.speculated_unit_type

dat_sampling_rate = 30000.0
lfpSampleRate = 1250.0
posSampleRate = 29.9700

session.neurons = Neurons(spiketrains, t_stop, t_start=0,
    sampling_rate=dat_sampling_rate, # session.recinfo.dat_sampling_rate
    neuron_ids=flat_cell_ids,
    neuron_type=cell_type,
    shank_ids=shank_ids
)
session.neurons

# Neurons
#  n_neurons: 126
#  n_total_spikes: 11214527
#  t_start: 0
#  t_stop: 35351.059333

# 'num_cells': 86, 'spike_list', 'reverse_cellID_idx_lookup_map'

In [31]:
session

DataSession(config: <SessionConfig: {'basepath': WindowsPath('R:/rMBP Python Repos 2022-07-07/PhoNeuronGillespie2021CodeRepo/PhoMatlabDataScripting/ExportedData/RoyMaze1'), 'session_name': 'RoyMaze1', 'session_spec': <SessionFolderSpec: {'required_files': [], 'optional_files': [], 'additional_validation_requirements': []};>, 'is_resolved': True, 'resolved_required_filespecs_dict': {}, 'resolved_optional_filespecs_dict': {}, 'absolute_start_timestamp': 68368.714228, 'position_sampling_rate_Hz': 29.97016853950917};>): Not yet configured.

In [ ]:
## IDEA: Define a series of fallback options for each variable using the command chain pattern.